In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from PIL import Image
import os
from functools import partial

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

import matplotlib
import matplotlib.pyplot as plt

In [4]:
# Helper functions

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:, :, 0] = gt_img8
        gt_img_3c[:, :, 1] = gt_img8
        gt_img_3c[:, :, 2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

In [9]:
data_path = "../data/training/"
train_path = data_path + "images/"
label_path = data_path + "groundtruth/"
image_path_list = os.listdir(train_path)

image_path=os.path.join(train_path, image_path_list[0])
label_pt=os.path.join(label_path, image_path_list[0])

image = Image.open(image_path)
label = Image.open(label_pt)

image_array = np.array(image)
label_array = np.array(label)

# fig, ax= plt.subplots(1,2)
# ax[0].imshow(image_array)
# ax[1].imshow(label_array)

In [10]:
from torch.utils.data import Dataset

class RoadSegmentationDataset(Dataset):
    def __init__(self, data_dir, image_folder = 'images', mask_folder = 'groundtruth', transform = None):
        """
        Args:
            data_dir (str): Root directory of the dataset.
            image_folder (str): Name of the folder containing road images.
            mask_folder (str): Name of the folder containing segmentation masks.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_dir = data_dir
        self.image_folder = os.path.join(data_dir, image_folder)
        self.mask_folder = os.path.join(data_dir, mask_folder)
        self.transform = transform

        assert os.path.exists(self.image_folder), f"Image folder '{self.image_folder}' not found."
        assert os.path.exists(self.mask_folder), f"Mask folder '{self.mask_folder}' not found."

        self.image_list = sorted(os.listdir(self.image_folder))
        self.mask_list = sorted(os.listdir(self.mask_folder))


    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):

        img_name = self.image_list[idx]
        mask_name = self.mask_list[idx]

        img_path = os.path.join(self.image_folder, img_name)
        mask_path = os.path.join(self.mask_folder, mask_name)

        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"))    

        means = torch.tensor([np.mean(image.astype(np.float32)[:,:,i]) for i in range(3)])
        std = torch.tensor([np.std(image.astype(np.float32)[:,:,i]) for i in range(3)])
        normalize = transforms.Normalize(mean = means, std = std)
        tens = torch.tensor(np.transpose(image_array.astype(np.float32), (2, 1, 0))).unsqueeze(0)
        normalized_input = normalize(tens)
        lab = np.expand_dims(mask/np.max(label_array), axis=0)
        mask = torch.tensor(lab.astype(np.float32)).unsqueeze(0)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return (normalized_input,mask)
        return img_tensor, label_tensor

In [11]:
dataset = RoadSegmentationDataset(data_dir=data_path, transform=None)

# Test the dataset initialization
sample_img, sample_label = dataset[0]
print(f"Sample Image Shape: {sample_img.shape}")
print(f"Sample Label Shape: {sample_label.shape}")

Sample Image Shape: torch.Size([1, 3, 400, 400])
Sample Label Shape: torch.Size([1, 1, 400, 400])


In [ ]:
class AlexNet(nn.module):
    